In [1]:
from IPython.display import GeoJSON
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings

## The code threw up some SettingWithCopyWarnings that I will fix at somepoint
warnings.filterwarnings('ignore')

In [2]:
sensorList = ['PER_AIRMON_MONITOR1048100',
                'PER_AIRMON_MONITOR1156100', 
                'PER_AIRMON_MONITOR1056100',
                'PER_AIRMON_MONITOR914', 
                'PER_AIRMON_MONITOR1155100', 
                'PER_AIRMON_MONITOR1135100']

friendlyNames = {
        'PER_AIRMON_MONITOR1048100': 'Coast Road @ Centurion Park',
        'PER_AIRMON_MONITOR1156100': 'Angel of the North',
        'PER_AIRMON_MONITOR1056100': 'Tyne Bridge: A167 Gateshead Side',
        'PER_AIRMON_MONITOR914': 'Gosforth: Gosforth Street - Salters Road',
        'PER_AIRMON_MONITOR1155100': 'Four Lane Ends: Front Street - Benton Road',
        'PER_AIRMON_MONITOR1135100': 'Central Newcastle: Pilgrim Street - Hood Street'
        }

## Key Variables - easy to add more plots
variables = ['NO2','PM2.5','PM10']

In [3]:
for s in sensorList:
    mainTitle = friendlyNames[s]
    ## Update filepath
    recentData = pd.read_pickle('C:/Covid_Baseline/recent_aq_'+s+'.pkl')

    recentData = recentData[recentData["Flagged as Suspect Reading"]==False]
    recentData.loc[:,"Weekday"] = pd.to_datetime(recentData['Timestamp']).dt.weekday
    recentData.loc[:,"Hour"] = pd.to_datetime(recentData['Timestamp']).dt.hour
    recentData.drop(columns=['Flagged as Suspect Reading','Location (WKT)','Ground Height Above Sea Level',
                            'Sensor Height Above Ground','Broker Name','Third Party','Sensor Centroid Longitude',
                            'Sensor Centroid Latitude','Raw ID'], axis=1, inplace=True)
    
    nrows = len(variables)
    fig, axs = plt.subplots(nrows,1, figsize=(30,30))
    row=0
    
    for v in variables:
        ## Update filepath
        dfBaseTemp = pd.read_pickle('C:/Covid_Baseline/baseline_aq_'+s+'_'+v+'.pkl')
        
        dfCovidTemp = recentData[recentData["Variable"]==v]
        dfCovidTemp.drop(columns=['Sensor Name','Variable','Units','Timestamp'], axis=1, inplace=True)
        
        # Aggregate Covid Data
        aggregateColumns = ['Weekday', 'Hour']
        dfCovidMean = dfCovidTemp.groupby(aggregateColumns, group_keys=False, as_index=False).median()
        dfCovidLQ = dfCovidTemp.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(.15)
        dfCovidHQ = dfCovidTemp.groupby(aggregateColumns, group_keys=False, as_index=False).quantile(.85)
        
        plt.axes(axs[row])
        plt.title(mainTitle + ' : ' + v, fontsize=23)
        plt.xlim(0,167)
        plt.xlabel('Day/Hour', fontsize=18)
        plt.xticks(ticks=[0,12,24,36,48,60,72,84,96,108,120,132,144,156,168], 
                   labels=['Mon 00','Mon 12','Tues 00','Tues 12','Wed 00','Wed 12','Thurs 00','Thurs 12',
                           'Fri 00','Fri 12','Sat 00','Sat 12','Sun 00','Sun 12'], fontsize=16)

        plt.ylabel(v, fontsize=18)
        plt.plot(dfBaseTemp["Mean"], color = "darkred",label="Baseline Median (Nov-Feb)")
        plt.plot(dfBaseTemp["LQ"], color = "salmon",linestyle='dashed',label="Baseline P15-P85 (Nov-Feb)")
        plt.plot(dfBaseTemp["HQ"], color = "salmon",linestyle='dashed')
        plt.plot(dfCovidMean["Value"], color = "navy",label="Post 16th March Median")
        plt.plot(dfCovidLQ["Value"], color = "skyblue",linestyle='dashed',label="Post 16th March P15-P85 (Nov-Feb)")
        plt.plot(dfCovidHQ["Value"], color = "skyblue",linestyle='dashed')
        
        plt.legend(loc="upper right",fontsize=18)
    
        row = row+1
         
    fig.tight_layout()
    ## Update filepath
    plt.savefig('C:/Covid_Baseline/Images/'+s+'_TS.png')
    plt.clf()

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>

<Figure size 2160x2160 with 0 Axes>